# Valley Bottom Extraction
## Iterate over regional outputs and select input datasets for RVBD tool as well as convert/create necessary datasets.

Create data dictionary to store input parameters for tool.
1. River Network
2. Catchments
3. DEM
4. Wetlands - as feature class or shapefile
5. Output Folder - Directory
6. River Valley Bottom - Output River Valley Bottom Polygon

In [31]:
# Convert wetlands to shapefile
import arcpy, os, sys

# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

# folder containing regional watersheds and covariate source data
datadir = r'D:\\GIS\\AKSSF'
arcpy.env.overwriteOutput = True
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
regions = arcpy.ListWorkspaces(workspace_type="Folder")
#working gdb to save intermediate outputs
scratchgdb = r"D:\\GIS\\scratch.gdb"
scratchfol = r"D:\\GIS\\scratch"

rvbdDict = {}
# Walk through gdbs and select wetlands raster to convert
for region in regions:
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, datatype=['RasterDataset','FeatureClass'])
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if 'streams_merge' == filename:
                strsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,strsource)
            # Set merged watersheds dataset
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,wtdsource)
            elif 'elev_tif' == filename:
                elev_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,elev_path)
            # Grab wetland raster
            elif filename == 'wetlands.tif':
                print(f'wetlands fc will be saved to {outgdb}')
                print(f'{dirpath},{filename}')
                wras_path = os.path.join(dirpath, filename)
                wrasname = filename[:-4]
                wetfcpath = os.path.join(outgdb, wrasname)
                wetfcpath2 = os.path.join(scratchgdb, wrasname)
                # Make local copy projected in AKAlbers
                if not arcpy.Exists(wetfcpath):
                    print(f'Converting {wrasname} to feature class...')
                    print('----------')
                    wetfc_conv = arcpy.RasterToPolygon_conversion(wras_path,wetfcpath2,raster_field='VALUE',
                                                             simplify='NO_SIMPLIFY',
                                                             create_multipart_features='MULTIPLE_OUTER_PART')
                    wetfc = arcpy.FeatureClassToFeatureClass_conversion(wetfcpath2,outgdb,wrasname,
                                                                        where_clause='gridcode = 1')
                else:
                    print(f'{wetfcpath} already created')
                    wetfc = wetfcpath
                    print('----------')
                append_value(rvbdDict,rname,wetfcpath)
                append_value(rvbdDict,rname,scratchfol)
                rivbottpath = os.path.join(dirpath + '.gdb','riv_valbot')
                append_value(rvbdDict,rname,rivbottpath)

Bristol_Bay
wetlands fc will be saved to D:\\GIS\\AKSSF\Bristol_Bay\Bristol_Bay.gdb
D:\\GIS\\AKSSF\Bristol_Bay,wetlands.tif
D:\\GIS\\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wetlands already created
----------
Cook_Inlet
wetlands fc will be saved to D:\\GIS\\AKSSF\Cook_Inlet\Cook_Inlet.gdb
D:\\GIS\\AKSSF\Cook_Inlet,wetlands.tif
D:\\GIS\\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wetlands already created
----------
Copper_River
wetlands fc will be saved to D:\\GIS\\AKSSF\Copper_River\Copper_River.gdb
D:\\GIS\\AKSSF\Copper_River,wetlands.tif
D:\\GIS\\AKSSF\Copper_River\Copper_River.gdb\wetlands already created
----------
Kodiak
wetlands fc will be saved to D:\\GIS\\AKSSF\Kodiak\Kodiak.gdb
D:\\GIS\\AKSSF\Kodiak,wetlands.tif
D:\\GIS\\AKSSF\Kodiak\Kodiak.gdb\wetlands already created
----------
Prince_William_Sound
wetlands fc will be saved to D:\\GIS\\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb
D:\\GIS\\AKSSF\Prince_William_Sound,wetlands.tif
D:\\GIS\\AKSSF\Prince_William_Sound\Prince_William_Sound.g

In [32]:
#Check paths in dictionary
rvbdDict

{'Bristol_Bay': ['D:\\\\GIS\\\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\\\GIS\\\\AKSSF\\Bristol_Bay.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wtds_merge'],
 'Cook_Inlet': ['D:\\\\GIS\\\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\\\GIS\\\\AKSSF\\Cook_Inlet.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wtds_merge'],
 'Copper_River': ['D:\\\\GIS\\\\AKSSF\\Copper_River\\Copper_River.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\\\GIS\\\\AKSSF\\Copper_River.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\wtds_merge'],
 'Kodiak': ['D:\\\\GIS\\\\AKSSF\\Kodiak\\Kodiak.gdb\\wetlands',
  'D:\\\\GIS\\\\scratch',
  'D:\\\\GIS\\\\AKSSF\\Kodiak.gdb\\riv_valbot',
  'D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wtds_merge'],
 'Prince_William_Sound': [

In [34]:
#Begin River Valley Bottom tool
rvbd_dir = r"D:\\RVBD\\"
# Import modules
import sys
import arcpy
import os
from pathlib import Path
import datetime
import sys


# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0] / "py")
print(module_path)

# # in standard python
# module_path = str(Path.cwd(__file__).parents[0] / "py")

if module_path not in sys.path:
    sys.path.append(module_path)

C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers\py


In [ ]:
from modules import preparations
import tools
from employee import motivation_to_work
# from slope import slope
from valleycostdistance import valley
from costdistance import costdist


In [ ]:
# Starting script
start = datetime.datetime.now()
print("Script started at %s" % start)
arcpy.AddMessage ("Script started started at %s" % start)
arcpy.AddMessage ("\n")

# Check out spatial extention
arcpy.CheckOutExtension("Spatial")

# Environments
arcpy.env.overwriteOutput = True

# Python inputs
riv =  r"D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\streams_merge" # river network as a shapefile
cat = r"D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wtds_merge" # catchments as a shapefile
dem = r"D:\\GIS\\AKSSF\\Bristol_Bay\\elev.tif"# elevation model as a raster
wet = r"D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wetlands" # measured valley bottom, wetlands or any wet signature in low areas
pth = r"D:\\GIS\\scratch" # path to outputs (scratch and cost distance folders)
outval = r"D:\\GIS\\scratch.gdb\bbay_rvbd test" # path to output river valley
sloperas = r"D:\\GIS\\AKSSF\\Bristol_Bay\\slope.tif"

# # ArcGIS inputs
# riv =  arcpy.GetParameterAsText(0) # river network as a shapefile
# cat = arcpy.GetParameterAsText(1) # catchments as a shapefile
# dem = arcpy.GetParameterAsText(2) # elevation model as a raster
# wet = arcpy.GetParameterAsText(3) # measured river valley
# pth = arcpy.GetParameterAsText(4) # path to outputs (scratch and cost distance folders)
# outval = arcpy.GetParameterAsText(5) # path to output river valley

# Check to see if scratch geodatabase exists
s = Path(os.path.join(pth, "outputs.gdb"))
if s.exists() == False:
    scrg = arcpy.CreateFileGDB_management(pth, "outputs")
    scr = scrg.getOutput(0) # scratch folder
else:
    scr = os.path.join(pth, "outputs.gdb") #scr = r"in_memory" (if you want to store in memory)

# Check to see if cost dist folder exists (use code in comments if you want results in a folder instead of gdb)
c = Path(os.path.join(pth, "costdist.gdb")) # Path(os.path.join(pth, "costdist"))
if c.exists() == False:
    cstg = arcpy.CreateFileGDB_management(pth, "costdist")  # arcpy.CreateFolder_management(pth, "costdist")
    cst = cstg.getOutput(0) # cost distance folder
else:
    cst = os.path.join(pth, "costdist.gdb") # os.path.join(pth, "costdist")

# Calculate slope and replace 0 slopes with small value - We already have slope rasters so no need for this step
# slope(riv, cat, dem, scr)

# Cost distance calculation
costdist(riv, cat, sloperas, cst, scr)

# Extract valley
valley(riv, wet, cst, scr, outval)

# Ending script
end = datetime.datetime.now()
print("Script ended at %s" % end)
arcpy.AddMessage ("Script ended at %s" % end)
arcpy.AddMessage ("\n")
time_elapsed = end - start
print("Time elapsed %s" % (time_elapsed))
arcpy.AddMessage ("Time elapsed %s" % time_elapsed)
arcpy.AddMessage ("\n")
print(".........................................................")
arcpy.AddMessage (".........................................................")
arcpy.AddMessage ("\n")


